# association rules e fpgrowth

In [1]:
import fim
from fim import fpgrowth
from tqdm import tqdm
import numpy as np
import scipy.sparse as sps
import numpy as np
import scipy.sparse as sps
from collections import defaultdict
from fim import apriori, fpgrowth, fim, arules
from utils.post_processing import rec_list_to_eurm,eurm_to_recommendation_list,eurm_remove_seed
from utils.evaluator import Evaluator
from utils.datareader import Datareader
from utils.definitions import *
from multiprocessing import Process, Pool


In [2]:

def build_test_dict(dr):
    print("building test dict", end=" ")
    test_interactions_df = dr.get_df_test_interactions()
    test_interactions_df.sort_values(['pos'], ascending=True)
    test_playlists_df = dr.get_df_test_playlists()
    test_playlists = test_playlists_df['pid'].values

    # A list of list [pos, tid] for each playlist sorted by pos
    test_known_tracks = test_interactions_df.groupby(['pid'])[['pos', 'tid']].apply(lambda x: x.values.tolist())
    for s in test_known_tracks:
        s = s.sort(key=lambda x: x[0])
    print("> done")
    return test_known_tracks

def urm_to_sequences_from_one_target(urm_pos, urm_pos_csc, song_target, list_of_list_of_lists=False):
    not_empty_lines = urm_pos_csc[:,song_target].nonzero()[0] 
    filtered = urm_pos[not_empty_lines]
    sequences_spm = []
    for row in range(filtered.shape[0]):
        to_append = list(filtered.indices[filtered.indptr[row]:filtered.indptr[row + 1]]
                             [np.argsort(filtered.data[filtered.indptr[row]:filtered.indptr[row + 1]])])
        if list_of_list_of_lists:
            sequences_spm.append( [[i] for i in to_append])
        else:
            sequences_spm.append(to_append)
    return sequences_spm

def fast_argpart(arr):
    if len(arr)>500:
        max_n = 500
    else: 
        max_n = len(arr)
    return np.argpartition(arr, -max_n)[-max_n:]
    

# datie roba varia

In [3]:
dr = Datareader(mode='offline', verbose=False, only_load=True)
ev = Evaluator(dr)

/home/ubuntu/Spotify-Challenge/utils/evaluator.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  keys = list(self.tracks['tid'].as_matrix())
/home/ubuntu/Spotify-Challenge/utils/evaluator.py:29: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  values = list(self.tracks['arid'].as_matrix())


In [4]:
test_known_tracks = build_test_dict(dr)
test_pids_cat2 = dr.get_test_pids(cat=2)

building test dict > done


In [5]:
urm_pos = dr.get_position_matrix(position_type='last')
urm_pos_csc = sps.csc_matrix(urm_pos)

# creazione dei file. lenta e da non rifare 

In [6]:
###### NON FARE QUESTA CELLA
# for i in tqdm(range(1700,1800)):

#     song_target = test_known_tracks[test_pids_cat2[i-1000]][0][1]
#     not_empty_lines = urm_pos_csc[:,song_target].nonzero()[0] 
#     filtered = urm_pos[not_empty_lines]
#     sequences_spm = []
#     for row in range(filtered.shape[0]):
#         to_append = list(filtered.indices[filtered.indptr[row]:filtered.indptr[row + 1]]
#                              [np.argsort(filtered.data[filtered.indptr[row]:filtered.indptr[row + 1]])])
#         sequences_spm.append(to_append)
#     save_obj(name="sequences_cat1_"+str(i), obj=sequences_spm, path=ROOT_DIR+'/data/cat1/'  )

# association rules versione normale


In [7]:
costante_di_popolarita = 15

In [8]:
# pred_lil = sps.lil_matrix((10000, 2262292))
# for i in tqdm(range(1000,1070)):
#     sequences = load_obj(path=ROOT_DIR+'/data/cat1/', name='sequences_cat1_'+str(i))
#     popularity = len(sequences)
#     preds_line = np.zeros(2262292)
    
#     for seq in fpgrowth(sequences,supp= -popularity/costante_di_popolarita, target='m'):
#         for song in seq[0]:
#             preds_line[song]+= seq[1]*(len(seq[0])-1)*(len(seq[0])-1)
#     vals = fast_argpart(preds_line)
    
#     pred_lil[i,vals] = preds_line[vals]


In [9]:
# eurm = sps.csr_matrix(pred_lil)
# eurm = eurm_remove_seed(eurm , dr )
# rec_list = eurm_to_recommendation_list(eurm)
# ev.evaluate(rec_list, "cat2_spm_max",verbose=True, do_plot=True, show_plot=True, save=True )

# parallel association rule. 

In [8]:
import gc

target = 'm'
costante_di_pop = 15

In [9]:
def association_rule(i):
    sequences = load_obj(path=ROOT_DIR+'/data/cat1/', name='sequences_cat1_'+str(i))
    popularity_iniziale = len(sequences)
    preds_line = np.zeros(2262292)
    
    if popularity_iniziale>2000:
        mean_len = 0 
        for seq in sequences:
            mean_len += len(seq)
        mean_len= mean_len/len(sequences)
        
        count = 0
        for j in range(len(sequences)):
            if len(sequences[j]) > (mean_len*2) or len(sequences[j]) <(mean_len/2):
                sequences[j]=[]
                count+=1
        popularity = popularity_iniziale-count
        
        print(i,"iniziale"popularity_iniziale,"new_pop",popularity,"rimosse",count," mean_l",mean_len,"num_seq",len(sequences))
    
        if popularity>2000:
            mean_len = 0 
            for seq in sequences:
                mean_len += len(seq)
            mean_len= mean_len/len(sequences)

            count = 0
            for j in range(len(sequences)):
                if len(sequences[j]) > (mean_len*2) or len(sequences[j]) <(mean_len/2):
                    sequences[j]=[]
                    count+=1
            popularity-=count

            print(i,popularity_iniziale,"new_pop",popularity,"rimosse",count," mean_l",mean_len,"num_seq",len(sequences))

        if popularity>2000:
            mean_len = 0 
            for seq in sequences:
                mean_len += len(seq)
            mean_len= mean_len/len(sequences)

            count = 0
            for j in range(len(sequences)):
                if len(sequences[j]) > (mean_len*2) or len(sequences[j]) <(mean_len/2):
                    sequences[j]=[]
                    count+=1
            popularity-=count
            print(i,popularity_iniziale,"new_pop",popularity,"rimosse",count," mean_l",mean_len,"num_seq",len(sequences))
    

#     else:
#         if popularity>50000:
#             const = costante_di_pop*10
#         elif popularity>30000:
#             const = costante_di_pop*5
#         elif popularity>20000:
#             const = costante_di_pop*3
#         else:
#             const = costante_di_pop*2
    sequences = np.array(sequences)
    sequences = sequences[len(sequences)>0]
    const = costante_di_pop

    sequences = fpgrowth(sequences, supp= -popularity/const, target=target)
    
    
    for seq in sequences:
        for song in seq[0]:
            preds_line[song]+= seq[1]*(len(seq[0])-1)*(len(seq[0])-1)
    indices = fast_argpart(preds_line)
    
    preds_line_lil = sps.lil_matrix((1, 2262292))
    vals = fast_argpart(preds_line)
    preds_line_lil[0,vals] = preds_line[vals]
    
    del sequences, indices, preds_line, vals, 
    gc.collect()
    print("nnz",preds_line_lil.nnz)
    
    return preds_line_lil


In [10]:
p = Pool(2)
pred_lil = sps.lil_matrix((10000, 2262292))
roba = p.map(association_rule, [x+1000 for x in range(1000)])


1125 2241 new_pop 956 rimosse 1285  mean_l 97.71396697902722 num_seq 2241
1000 6826 new_pop 3082 rimosse 3744  mean_l 109.39173747436273 num_seq 6826
1000 6826 new_pop -662 rimosse 3744  mean_l 74.28713741576325 num_seq 6826
nnz 199
nnz 0


UnboundLocalError: local variable 'popularity' referenced before assignment

1250 6856 new_pop 2995 rimosse 3861  mean_l 97.60822637106185 num_seq 6856
1250 6856 new_pop -866 rimosse 3861  mean_l 67.74868728121353 num_seq 6856
nnz 0
1375 3910 new_pop 1677 rimosse 2233  mean_l 93.30358056265985 num_seq 3910
1126 28713 new_pop 12712 rimosse 16001  mean_l 101.44133319402361 num_seq 28713
1126 28713 new_pop -3289 rimosse 16001  mean_l 70.38578344303974 num_seq 28713
nnz 190
1625 11619 new_pop 4843 rimosse 6776  mean_l 88.71667096996299 num_seq 11619
1625 11619 new_pop -1933 rimosse 6776  mean_l 61.2570789224546 num_seq 11619
nnz 0
nnz 0
1127 5594 new_pop 2414 rimosse 3180  mean_l 96.07883446549874 num_seq 5594
1127 5594 new_pop -766 rimosse 3180  mean_l 66.47604576331784 num_seq 5594
nnz 0
1128 3450 new_pop 1511 rimosse 1939  mean_l 95.64637681159421 num_seq 3450
nnz 170
1750 39573 new_pop 17427 rimosse 22146  mean_l 98.76554721653653 num_seq 39573
1750 39573 new_pop -4719 rimosse 22146  mean_l 68.53695701614737 num_seq 39573
nnz 0
1751 18700 new_pop 8001 rimosse 1

In [28]:
pred_lil = sps.lil_matrix((10000, 2262292))
for prog, preds in enumerate(roba):
    pred_lil[prog+1000]= preds

In [29]:
pred_lil

<10000x2262292 sparse matrix of type '<class 'numpy.float64'>'
	with 5786 stored elements in LInked List format>

In [30]:
eurm = sps.csr_matrix(pred_lil)
eurm = eurm_remove_seed(eurm , dr )
rec_list = eurm_to_recommendation_list(eurm)
ev.evaluate(rec_list, "cat2_spm_max",verbose=True, do_plot=True, show_plot=True, save=True )

Converting eurm:   2%|▏         | 214/10000 [00:00<00:04, 2138.52it/s]

[ WARNING! Datareader is None. It was not possible to remove seeds while converting the eurm ]


Evaluating: 100%|██████████| 10000/10000 [00:16<00:00, 616.85it/s]

       clicks_artists  clicks_tracks  ndcg_artists  ndcg_tracks  \
cat1          51.0000        51.0000      0.000000     0.000000   
cat2          49.5860        49.7590      0.009650     0.005837   
cat3          51.0000        51.0000      0.000000     0.000000   
cat4          51.0000        51.0000      0.000000     0.000000   
cat5          51.0000        51.0000      0.000000     0.000000   
cat6          51.0000        51.0000      0.000000     0.000000   
cat7          51.0000        51.0000      0.000000     0.000000   
cat8          51.0000        51.0000      0.000000     0.000000   
cat9          51.0000        51.0000      0.000000     0.000000   
cat10         51.0000        51.0000      0.000000     0.000000   
mean          50.8586        50.8759      0.000965     0.000584   

       precision_artists  precision_tracks  
cat1            0.000000          0.000000  
cat2            0.000503          0.003196  
cat3            0.000000          0.000000  
cat4           


/home/ubuntu/Spotify-Challenge/utils/evaluator.py:190: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  tracks_values = evaluation_df[metric + '_tracks'].as_matrix()
/home/ubuntu/Spotify-Challenge/utils/evaluator.py:191: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  artists_values = evaluation_df[metric + '_artists'].as_matrix()


(0.00031956564542555253,
 0.0005837060745494298,
 50.8759,
 5.033028710195894e-05,
 0.0009649505194430582,
 50.8586)

In [34]:
pred_lil = sps.lil_matrix((10000, 2262292))
for i in tqdm(range(1000,1030), desc='roba varia '):
    sequences = load_obj(path=ROOT_DIR+'/data/cat1/', name='sequences_cat1_'+str(i))
    popularity = len(sequences)
    preds_line = np.zeros(2262292)
    
    for seq in fpgrowth(sequences,supp= -popularity/costante_di_popolarita, target='c'):
        for song in seq[0]:
            preds_line[song]+= seq[1]*(len(seq[0])-1)*(len(seq[0])-1)
    vals = fast_argpart(preds_line)
    
    pred_lil[i,vals] = preds_line[vals]
eurm = sps.csr_matrix(pred_lil)
eurm = eurm_remove_seed(eurm , dr )
rec_list = eurm_to_recommendation_list(eurm)
ev.evaluate(rec_list, "cat2_spm_max",verbose=True, do_plot=True, show_plot=True, save=True )



roba varia :   0%|          | 0/30 [00:00<?, ?it/s]

roba varia :   3%|▎         | 1/30 [00:02<01:16,  2.63s/it]

roba varia :   7%|▋         | 2/30 [00:02<00:38,  1.38s/it]

roba varia :  17%|█▋        | 5/30 [00:02<00:14,  1.72it/s]

roba varia :  20%|██        | 6/30 [00:03<00:12,  1.98it/s]

roba varia :  23%|██▎       | 7/30 [00:03<00:11,  2.09it/s]

roba varia :  30%|███       | 9/30 [00:03<00:09,  2.26it/s]

roba varia :  33%|███▎      | 10/30 [00:04<00:08,  2.44it/s]

roba varia :  40%|████      | 12/30 [00:04<00:06,  2.70it/s]

roba varia :  43%|████▎     | 13/30 [00:04<00:06,  2.77it/s]

roba varia :  47%|████▋     | 14/30 [00:05<00:06,  2.66it/s]

roba varia :  50%|█████     | 15/30 [00:06<00:06,  2.46it/s]

roba varia :  53%|█████▎    | 16/30 [00:06<00:05,  2.44it/s]

KeyboardInterrupt: 

Converting eurm:   6%|▋         | 633/10000 [00:00<00:01, 6320.27it/s]

[ WARNING! Datareader is None. It was not possible to remove seeds while converting the eurm ]


Evaluating: 100%|██████████| 10000/10000 [00:06<00:00, 1636.53it/s]

       clicks_artists  clicks_tracks  ndcg_artists  ndcg_tracks  \
cat1          51.0000        51.0000      0.000000     0.000000   
cat2           8.5620        13.1590      0.278146     0.176737   
cat3          51.0000        51.0000      0.000000     0.000000   
cat4          51.0000        51.0000      0.000000     0.000000   
cat5          51.0000        51.0000      0.000000     0.000000   
cat6          51.0000        51.0000      0.000000     0.000000   
cat7          51.0000        51.0000      0.000000     0.000000   
cat8          51.0000        51.0000      0.000000     0.000000   
cat9          51.0000        51.0000      0.000000     0.000000   
cat10         51.0000        51.0000      0.000000     0.000000   
mean          46.7562        47.2159      0.027815     0.017674   

       precision_artists  precision_tracks  
cat1            0.000000          0.000000  
cat2            0.014166          0.098417  
cat3            0.000000          0.000000  
cat4           


/home/ubuntu/Spotify-Challenge/utils/evaluator.py:190: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  tracks_values = evaluation_df[metric + '_tracks'].as_matrix()
/home/ubuntu/Spotify-Challenge/utils/evaluator.py:191: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  artists_values = evaluation_df[metric + '_artists'].as_matrix()


(0.00984173390443778,
 0.017673749302371637,
 47.2159,
 0.001416606671301035,
 0.02781461782245947,
 46.7562)